# OpenCV Simple example
going through some simple tests to make sure the camera works as expected
This is using the `opencv` library, install via pip with `opencv-python`


In [1]:
'''
Does the camera work correctly? 

Basically look for "opencv camera tutorial" on your platform and start from there.
'''
import cv2
import numpy as np

# I know it's '2' from before
interface=2
camera = None
camera = cv2.VideoCapture(interface)

while True:
    _, img = camera.read()
    if img is None:
        print(f"Unable to open camera {interface=}")
        break
    cv2.imshow("simple camera test", img)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()

In [2]:
# Part 0 - breaking up the image

while True: 
    _, imag = camera.read()
    if img is None:
        print(f"Unable to open camera {interface=}")
        break
    # split image
    b_img = img[:,:,0]
    g_img = img[:,:,1]
    r_img = img[:,:,2]
    # stack next to each other
    hz = np.hstack((b_img,g_img,r_img))
    cv2.imshow('3 Stack', hz)
    if cv2.waitKey(1) == 27:
        break

cv2.destroyAllWindows()

# Part 1 - Thresholding

```python
for pixel in image:
    if pixel.value > thresh:
        pixel = BLACK
    else:
        pixel = WHITE
```